In [ ]:

# # cuml Random Forest params
# cu_rf_params = {
#     ‘n_estimators’: 25,
#     ‘max_depth’: 13,
#     ‘n_bins’: 15,
#     ‘n_streams’: 8
# }

# # Start by setting up the CUDA cluster on the local host 
# cluster = LocalCUDACluster(threads_per_worker=1, n_workers=n_workers)
# c = Client(cluster)
# workers = c.has_what().keys()

# # Shard the data across all workers
# X_train_df, y_train_df = dask_utils.persist_across_workers(c,[X_train_df,y_train_df],workers=workers)

# # Build and train the model
# cu_rf_mg = cuRFC_mg(**cu_rf_params)
# cu_rf_mg.fit(X_train_df, y_train_df)

# # Check the accuracy on a test set
# cu_rf_mg_predict = cu_rf_mg.predict(X_test)
# acc_score = accuracy_score(cu_rf_mg_predict, y_test, normalize=True)
# c.close()
# cluster.close()

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix

In [2]:
y_train, _, x_train = pickle.load(open("dbpedia_csv/" + "/train_vec_preprocessed_vectorized.pickle", "rb"))
#_, _, x_valid = pickle.load(open(paths.data + "/validation_preprocessed.pickle", "rb"))
y_test, _, x_test = pickle.load(open("dbpedia_csv/" + "/test_vec_preprocessed_vectorized.pickle", "rb"))

y_train = y_train[slice(0, len(y_train), 4)]
x_train = x_train[slice(0, len(x_train), 4)]

In [5]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
forest = RandomForestClassifier(n_estimators = 100,verbose=3,n_jobs=-1, max_depth=15)

In [8]:
forest = forest.fit(x_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100

building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   36.8s


building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.3min finished


In [10]:
y_pred = forest.predict(x_test)
score1 = accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(classification_report(y_test, y_pred, target_names=[str(x) for x in range(14)]))

print("confusion matrix:")
print(confusion_matrix(y_test, y_pred))

print('------------------------------')

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    1.0s finished


accuracy:   0.943
              precision    recall  f1-score   support

           0       0.89      0.86      0.87      5000
           1       0.95      0.96      0.95      5000
           2       0.87      0.82      0.84      5000
           3       0.98      0.98      0.98      5000
           4       0.94      0.95      0.95      5000
           5       0.95      0.97      0.96      5000
           6       0.94      0.93      0.94      5000
           7       0.97      0.98      0.97      5000
           8       1.00      0.97      0.99      5000
           9       0.98      0.98      0.98      5000
          10       0.98      0.98      0.98      5000
          11       0.94      0.97      0.95      5000
          12       0.95      0.95      0.95      5000
          13       0.85      0.91      0.88      5000

    accuracy                           0.94     70000
   macro avg       0.94      0.94      0.94     70000
weighted avg       0.94      0.94      0.94     70000

confusi

In [11]:
random_grid = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [50, 100, 150, 200]}

from sklearn.model_selection import cross_val_score

# scores = cross_val_score(forest, X, y, cv=3)

from sklearn.model_selection import RandomizedSearchCV

rf_random = RandomizedSearchCV(estimator = forest, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [12]:
rf_random.fit(x_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 200building tree 2 of 200
building tree 3 of 200
building tree 4 of 200

building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.1min


building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200
building tree 44 of 200
building tree 45 of 200
building tree 46 of 200
building tree 47 of 200
building tree 48 of 200
building tree 49 of 200
building tree 50 of 200
building tree 51 of 200
building tree 52 of 200
building tree 53 of 200
building tree 54 of 200
building tree 55 of 200
building tree 56 of 200
building tree 57 of 200
building tree 58 of 200
building tree 59 of 200
building tree 60 of 200
building tree 61 of 200
building tree 62 of 200
building tree 63 of 200
building tree 64 of 200
building tree 65 of 200
building tree 66

[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  7.1min


building tree 120 of 200
building tree 121 of 200
building tree 122 of 200
building tree 123 of 200
building tree 124 of 200
building tree 125 of 200
building tree 126 of 200
building tree 127 of 200
building tree 128 of 200
building tree 129 of 200
building tree 130 of 200
building tree 131 of 200
building tree 132 of 200
building tree 133 of 200
building tree 134 of 200
building tree 135 of 200
building tree 136 of 200
building tree 137 of 200
building tree 138 of 200
building tree 139 of 200
building tree 140 of 200
building tree 141 of 200
building tree 142 of 200
building tree 143 of 200
building tree 144 of 200
building tree 145 of 200
building tree 146 of 200
building tree 147 of 200
building tree 148 of 200
building tree 149 of 200
building tree 150 of 200
building tree 151 of 200
building tree 152 of 200
building tree 153 of 200
building tree 154 of 200
building tree 155 of 200
building tree 156 of 200
building tree 157 of 200
building tree 158 of 200
building tree 159 of 200


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 12.3min finished


RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(max_depth=15, n_jobs=-1,
                                                    verbose=3),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 150, 200]},
                   random_state=42, verbose=2)

In [13]:
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 40,
 'bootstrap': False}

In [15]:
rf_random.cv_results_

{'mean_fit_time': array([1654.4527541 , 1515.88813043, 1544.25133491, 2062.31285723,
        1973.19673506,  731.42882331, 1223.00728528,  392.11147428,
         871.13370752,  495.20545642, 1519.03530471, 1965.79785641,
         650.54773116,  315.38499125, 1425.49013646, 1199.19079852,
         653.30388149,  690.64967259,  565.77215028,  625.13191398,
         964.6976556 ,  860.00702802,  859.74155196,  336.82941914,
        1539.23209675,  319.80119602, 1155.90723689,  906.00362221,
         313.38420065,  565.69293634,  521.35683862, 1604.49888062,
        2224.66274818, 4194.10673213, 1770.28783162,  832.17639923,
         771.05411339, 1213.08220832,  857.45973349,  750.66713333,
        2860.59611734, 1723.32399249, 2414.88555694, 1121.49328375,
         860.00471425, 1090.52791548,  957.25025447, 1293.01085528,
         601.75237449, 2077.9167343 , 1223.4898146 ,  953.59275349,
        1440.23797846,  673.0741303 , 1294.37209328,  611.90596875,
         382.71951842, 1951.437

In [16]:
rf_random.best_score_

0.9482928595865779

In [28]:
rf_random.cv_results_['params'][22]

{'n_estimators': 100,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 20,
 'bootstrap': False}

In [35]:
rf_random.best_index_

93

In [36]:
rf_random.cv_results_['params'][93]

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 40,
 'bootstrap': False}

In [32]:
rf_random.cv_results_['params'][3]

{'n_estimators': 150,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': False}

In [31]:
rf_random.cv_results_['params'][57]

{'n_estimators': 200,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 20,
 'bootstrap': False}

In [34]:
rf_random.cv_results_['params'][4]

{'n_estimators': 150,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 20,
 'bootstrap': False}

In [ ]:
from sklearn.model_selection import GridSearchCV# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}# Create a based model

rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)